In [1]:
!pip install scikit-learn

In [2]:
!pip install joblib


In [3]:
!pip install wikipedia-api


In [4]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=26ced596e2e4332160511a91d50bbf0a89b131c97d3a9d9bfc1f755ca7172ecf
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [5]:
!pip install serpapi

In [6]:
!pip install -q transformers

In [7]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=da941f4a71e4a01744661977b527aa8a029b222fe322dc19c586748d0f4c30df
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [8]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html

Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 71.7 MB/s eta 0:00:00


In [9]:
import torch
torch.__version__

'2.1.0+cu121'

In [10]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pandas as pd
import numpy as np
from transformers import pipeline
import pandas as pd


# Training and testing Knn classifier

In [11]:
df = pd.read_csv('/content/topindplacelast.csv')


X = df[['Zone','State','Type','Significance', 'Best Time to visit',]]
y = df['City']


encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X.values).toarray()

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier

knn = KNeighborsClassifier(n_neighbors=5,metric='manhattan')


knn.fit(X_train, y_train)

from sklearn.metrics import accuracy_score


y_pred = knn.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')


Accuracy: 0.8770949720670391


In [15]:
def recommend_city(zone=None, state=None, type=None, significance=None, time_to_visit=None):

    df=pd.read_csv('/content/topindplacelast.csv')

    X = df[['Zone','State','Type','Significance', 'Best Time to visit']]
    y = df['City']


    encoder = OneHotEncoder()
    X_encoded = encoder.fit_transform(X).toarray()


    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    if state is None and zone is None:
        states = df['State'].unique()
        state = np.random.choice(states)

    if state is None or state not in df['State'].unique():
        states = df[df['Zone'] == zone]['State'].unique()
        state = np.random.choice(states)

    if zone is None:
        zone = df[df['State'] == state]['Zone'].unique()[0]

    if type is None or type not in df['Type'].unique():
        types = df['Type'].unique()
        type = np.random.choice(types)

    if significance is None or significance not in df['Significance']:
        significances = df['Significance'].unique()
        significance = np.random.choice(significances)


    if time_to_visit is None or time_to_visit not in df['Best Time to visit']:
        times = df['Best Time to visit'].unique()
        time_to_visit = np.random.choice(times)

    new_input = pd.DataFrame([[zone, state, type, significance, time_to_visit]], columns=['Zone', 'State', 'Type', 'Significance', 'Best Time to visit'])
    new_input_encoded = encoder.transform(new_input.values).toarray()

    prediction = knn.predict(new_input_encoded)
    predicted_city = label_encoder.inverse_transform(prediction)[0]
    places = df.loc[(df['City'] == predicted_city) & (df['Type'] == type), 'Name'].unique()
    return predicted_city , places





zone_input="Southern"
state_input=None
type_input="Beach"
significance_input=None
time_input=None

city,placelist=recommend_city(zone_input,state_input,type_input,significance_input,time_input)

print(city)
print(placelist.tolist())



Chennai
['Marina Beach', "Elliot's Beach", 'Breezy Beach', 'Elliots Beach', 'Covelong Beach', 'Besant Nagar Beach', 'Thiruvanmiyur Beach']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [24]:
from joblib import dump


model_filename = 'travel_model87updated.joblib'
dump(knn, model_filename)
from joblib import load

knn_loaded = load('/content/travel_model87updated.joblib')


Model saved to travel_model87updated.joblib


# functions used for flask file

In [16]:
import wikipedia
import pandas as pd
from serpapi import GoogleSearch
import requests

def fetch_weather(city):
    api_key = "1a531c047d464eb6b4941425242502"

    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}&aqi=no"


    response = requests.get(url)
    weather_info = {}

    if response.status_code == 200:

        data = response.json()


        location = data['location']['name']
        weather_description = data['current']['condition']['text']
        temperature = data['current']['temp_c']
        humidity = data['current']['humidity']
        wind_speed = data['current']['wind_kph']


        weather_info = {
            "Location": city,
            "Description": weather_description,
            "Temperature": f"{temperature}°C",
            "Humidity": f"{humidity}%",
            "Wind Speed": f"{wind_speed} km/h"
         }
    else:

        print(f"Failed to fetch weather data for {city}.")

    return weather_info

def getimages(place,city,state):
  api_key = 'c77f4ebdffaeb0ca01035ce5c49b2b5c5d998e37591616d07f0e0ec4691b7082'
  search=GoogleSearch({
      'engine': 'google_maps',
      'type': 'search',
      'q' : f'{place} , {city} , {state}',
      'api_key': 'c77f4ebdffaeb0ca01035ce5c49b2b5c5d998e37591616d07f0e0ec4691b7082',
  })
  revsa=[]
  imsa=[]
  try:
    results = search.get_dict()

    reviews=results['place_results']
    photo=reviews["thumbnail"]
    images=reviews["images"]
    for image in images:
      imsa.append(image['thumbnail'])

    user_reviews=reviews["user_reviews"]
    revs=user_reviews["summary"]
    for rev in revs:
      revsa.append(rev['snippet'])

    return imsa , revsa
  except:
    print(f'Not sufficent data found for {place}')

    return imsa , revsa


def extract_description(text):
    parts = text.split("\n")
    if parts:
        return parts[0]
    return text



def fetch_descriptions_and_save(places):
    data = []
    df=pd.read_csv('/content/topindplacelast.csv')

    wikipedia.set_lang("en")

    for place in places:
        try:

            filtered_df = df[df['Name'] == place]
            rating = str(filtered_df['Google review rating'].iloc[0])
            city = str(filtered_df['City'].iloc[0])
            state = str(filtered_df['State'].iloc[0])
            page = wikipedia.page(place)
            description = extract_description(page.content)
            imageslist,reviewslist = getimages(place,city,state)
            weatherdict = fetch_weather(city)
        except wikipedia.exceptions.PageError:
            description = "No description available"
        except wikipedia.exceptions.DisambiguationError:

            description = "Multiple pages found, specify further."


        data.append([place, description,reviewslist,imageslist,weatherdict,rating])


    df = pd.DataFrame(data, columns=['Name', 'Description','Reviews','Images','Weather','Rating'])

    return df



description_df = fetch_descriptions_and_save(placelist.tolist())




Not sufficent data found for Marina Beach
Not sufficent data found for Elliot's Beach
Not sufficent data found for Covelong Beach
Not sufficent data found for Besant Nagar Beach
Not sufficent data found for Thiruvanmiyur Beach


In [17]:
description_df

,Name,Description,Reviews,Images,Weather,Rating
0,Marina Beach,"Marina Beach, or simply the Marina, is a natur...",[],[],"{'Location': 'Chennai', 'Description': 'Partly...",3.9
1,Elliot's Beach,"Edward Elliot's Beach, simply called as Elliot...",[],[https://lh5.googleusercontent.com/p/AF1QipOml...,"{'Location': 'Chennai', 'Description': 'Partly...",4.2
2,Breezy Beach,No description available,[],[https://lh5.googleusercontent.com/p/AF1QipOml...,"{'Location': 'Chennai', 'Description': 'Partly...",4.2
3,Elliots Beach,No description available,[],[https://lh5.googleusercontent.com/p/AF1QipOml...,"{'Location': 'Chennai', 'Description': 'Partly...",4.4
4,Covelong Beach,Covelong Beach is actually Kovalam beach that ...,[],[],"{'Location': 'Chennai', 'Description': 'Partly...",4.3
5,Besant Nagar Beach,"Besant Nagar is an upscale, exclusive resident...",[],[https://lh5.googleusercontent.com/p/AF1QipOml...,"{'Location': 'Chennai', 'Description': 'Partly...",4.4
6,Thiruvanmiyur Beach,Thiruvanmiyur is a largely residential neighbo...,[],[https://lh5.googleusercontent.com/p/AF1QipOIt...,"{'Location': 'Chennai', 'Description': 'Partly...",4.2


# Loading and testing Document Q&A chat bot

In [18]:

from transformers import pipeline
import pandas as pd


def get_tqa_answer(query, table):

    table = table.astype(str)

    tqa_pipeline = pipeline(task="table-question-answering", model="google/tapas-base-finetuned-wtq")

    answer = tqa_pipeline(table=table, query=query)["answer"]

    return answer


table = description_df
table=table[:10]
query = "I like crowded beaches"
print("You Should Go to " + get_tqa_answer(query, table))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

You Should Go to Marina Beach, Besant Nagar Beach
